# Effect of 11/09 in NY flights 2000-2002

In [1]:
import gc
import glob
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

In [2]:
# for year in range(1987,2009):
#     filename = "{}.csv.bz2".format(year)
#     urllib.request.urlretrieve("http://stat-computing.org/dataexpo/2009/{}".format(filename),
#                                "../data/{}".format(filename))
#     print(year)

In [3]:
airports = pd.read_csv("../data/airports.csv.gz").query("state == 'NY'")
airports.head()

,iata,airport,city,state,country,lat,long
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
19,06N,Randall,Middletown,NY,USA,41.431566,-74.391917
42,0B8,Elizabeth,Fishers Island,NY,USA,41.251308,-72.031611
54,0G0,North Buffalo Suburban,Lockport,NY,USA,43.103184,-78.703346
57,0G7,Finger Lakes Regional,Seneca Falls,NY,USA,42.880623,-76.781620


In [4]:
df = pd.concat([pd.read_csv(x, index_col=None, encoding="iso-8859-1")
                for x in glob.glob("../data/200[0-2].csv.gz")], axis=0, ignore_index=True)
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647.0,1647,1906.0,1859,HP,154,...,15,11,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2000,1,29,6,1648.0,1647,1939.0,1859,HP,154,...,5,47,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2000,1,30,7,NaN,1647,NaN,1859,HP,154,...,0,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2000,1,31,1,1645.0,1647,1852.0,1859,HP,154,...,7,14,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2000,1,1,6,842.0,846,1057.0,1101,HP,609,...,3,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df[df.Dest.isin(airports.iata) | df.Origin.isin(airports.iata)].copy()
gc.collect()
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
4231,2000,1,1,6,1742.0,1740,2124.0,2120,DL,57,...,4,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4232,2000,1,2,7,1741.0,1740,2148.0,2120,DL,57,...,4,47,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4233,2000,1,3,1,1740.0,1740,2105.0,2120,DL,57,...,5,25,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4234,2000,1,4,2,1748.0,1740,2055.0,2120,DL,57,...,5,21,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4235,2000,1,5,3,1740.0,1740,2137.0,2120,DL,57,...,4,58,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [6]:
canc = df.groupby(["Year", "Month", "Cancelled"]).size().reset_index().set_index(["Year","Month"])\
         .query("Cancelled == 1").drop(['Cancelled'], axis=1).rename({0:"Cancelled"}, axis=1)
canc["Planned"] = df.groupby(["Year", "Month"]).size().reset_index().set_index(["Year","Month"])

In [7]:
import calendar
canc.reset_index(inplace=True)
canc.index =  canc.Month.apply(lambda x: calendar.month_abbr[x])+' '+ canc.Year.apply(str)
canc.index.name = "Date"
canc.drop(["Year","Month"], axis=1, inplace=True)
canc.reset_index(inplace=True)
canc.index.name = "Index"
canc

,Date,Cancelled,Planned
Index,,,
0,Jan 2000,3143,35178
1,Feb 2000,1480,33210
2,Mar 2000,884,35941
3,Apr 2000,1233,33854
4,May 2000,1864,35747
5,Jun 2000,2864,35279
6,Jul 2000,1903,37987
7,Aug 2000,2075,39148
8,Sep 2000,1583,37816


In [8]:
canc.to_csv("../data/cancelled.csv")